# Primary Text Processing

## Import

### Libraries

In [1]:
import os 
import codecs
from lxml import etree

from backend import *

### Definitions

In [2]:
texts = '../texts/fiction/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

## Primary Texts

### Library

In [3]:
libDf = pd.DataFrame(columns = libCols)
for t in os.listdir(texts): 
    if t[-4:] == '.txt': 
        #print(t)
        info = re.match(r'(\w+)-(\d{4})-(.+).txt', t)
        with codecs.open(texts+t, 'r', encoding='windows-1251') as f: 
            textytext = f.read()
        libDf = libDf.append({
            'author': info.group(1),
            'pub_year': int(info.group(2)), 
            'title': info.group(3), 
            'text': textytext
        }, ignore_index=True)
        
libDf = libDf.sort_values(libCols[1:3]).reset_index().drop(['index'], axis=1)
libDf.index.name = 'w_id'
libTextsDf = libDf[[libCols[3]]]
libDf = libDf.drop(columns=[libCols[3]])
libDf

,author,pub_year,title
w_id,,,
0,gorkii,1900,troe
1,andreev,1903,zhizn-vasiliia-fiveiskogo
2,andreev,1904,gubernator
3,andreev,1905,k-zvezdam
4,andreev,1905,khristiane
5,andreev,1905,tak-bylo
6,gorkii,1906,mat
7,andreev,1906,savva-ignis-sanat
8,andreev,1907,iuda-iskariot


In [4]:
def textRegularize(libTextDf, w_id):
    # grab text
    textDf = libTextsDf.iloc[[w_id]]
    if w_id in (6, 14):
        # split into chapters
        textDf = pd.DataFrame(data=textDf.text.str.split(r'\n\n').to_list()[0])# get chapter list
        chapTitles = textDf.iloc[::2][0].to_list()
        chapTexts = textDf.iloc[1::2][0].to_list()
        # add chapters to df
        textDf = pd.DataFrame(data={'chap':chapTitles, 'text':chapTexts})
        # clean chapter list of white space
        textDf.chap = textDf.chap.str.replace('\W', '', regex=True)
        # label parts
        textDf['part'] = ['1' if chap < 29 else '2' for chap in range(len(textDf.chap))]
        # break chapters into paragraphs
    textDf = textDf['text'].str.split(' \n', expand=True).stack().to_frame().reset_index().rename(columns={'level_0':'chapID','level_1':'para',0:'text'})
    #else:textDf = pd.DataFrame(data={'text':textDf.text.str.split(r'\n').to_list()[0]})
    # regularize
    textDf['text'] = textDf.text.str.replace('\n|\s{2,}', '')
    # remove white space paragraphs
    textDf = textDf.loc[~textDf.text.str.contains(r"^\W*$", regex=True)]
    #textDf['part'] = textDf.chapID.apply(lambda x: int('1') if x < 30 else int('2'))
    #textDf['chap'] = textDf.chapID.map(textDf['chapID'].to_dict())
    textDf['para'] = textDf['para'].apply(lambda x: x+1)
    textDf['paraID'] = range(1, len(textDf)+1)
    if w_id in (6, 14):
        textDf['chapID'] = textDf['chapID'].apply(lambda x: x+1)
        return textDf
    else:
        return textDf[['text', 'paraID']]
    
# make XML from text
def makeXML(textTitle, textDf, textXmlDf):
    root = etree.Element("text")
    print(root.tag)
    pt = ch = cn = pa = pn = 0
    nameDict = textDf.chap.to_dict()
    for chap in chapList:
        #print(f"Chap {chap}")
        root.append(etree.Element("chapter", n=str(cn+1), name=nameDict.get(chap)))
        paraList = textXmlDf.loc[(textXmlDf['part'] == part) & (textXmlDf['chapID'] == chap)].index
        #print(paraList)
        for paragraph in paraList:
            #print(f"Paragraph {paragraph}")
            root[ch].append(etree.Element("paragraph", n=str(pn+1), name=str(pa+1)))
            paraText = textXmlDf.loc[paragraph].text
            #print(f"paraText: {paraText}")
            #print(f"pt = {pt}; ch = {ch}; paragraph = {paragraph}")
            root[ch][pa].text = paraText
            pa+=1
            pn+=1
        pa=0
        ch+=1
        cn+=1
    #print(etree.tostring(root, pretty_print=True, xml_declaration=True))
    writePath = '..site/texts/'+textTitle+'.xml'
    etree.ElementTree(root).write(writePath, pretty_print=True, xml_declaration=True, encoding='windows-1251')

# make token Df from text
#motherTokenDf = nat_parse(motherXmlDf.set_index('paraID')[['text']])

In [5]:
confessionDf = textRegularize(libTextsDf, 10)
confessionDf

/tmp/ipykernel_420705/3670044570.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  textDf['text'] = textDf.text.str.replace('\n|\s{2,}', '')


,text,paraID
0,...Позвольте рассказать жизнь мою; времени пов...,1
1,"Я -- крапивник, подкидыш, незаконный человек; ...",2
2,"У Данилы прожил я до четырёх лет, но он сам мн...",3
3,"Четырёх лет взял меня к себе дьячок Ларион, че...",4
4,"От людей в стороне стоял, жил бедно, надел сво...",5
...,...,...
2357,Окрыляет он жизнь её величием деяний и чаяний ...,2356
2358,"-- Ты еси мой бог и творец всех богов, соткавш...",2357
2359,"-- Да не будут миру бози инии разве тебе, ибо ...",2358
2360,-- Тако верую и исповедую!,2359


In [6]:
dpDf = textRegularize(libTextsDf, 14)
dpDf

/tmp/ipykernel_420705/3670044570.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  textDf['text'] = textDf.text.str.replace('\n|\s{2,}', '')


,chapID,para,text,paraID
0,1,1,"Народ не только сила, создающая все материальн...",1
1,1,2,"Во дни своего детства, руководимый инстинктом ...",2
2,1,3,"В мифе и эпосе, как и в языке, главном деятеле...",3
3,1,4,Что образование и построение языка -- процесс ...,4
4,1,5,Мы еще не имеем достаточного количества данных...,5
...,...,...,...,...
287,12,28,"Но хулиган -- кровное дитя мещанина, это плод ...",266
288,12,29,Это драма -- семейная драма врага; мы смотрим ...,267
289,12,30,Нам -- это естественное желание здорового -- х...,268
290,12,31,Ибо для нас история всемирной культуры написан...,269


In [7]:
motherTextDf = textRegularize(libTextsDf, 6)
motherTextDf

/tmp/ipykernel_420705/3670044570.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  textDf['text'] = textDf.text.str.replace('\n|\s{2,}', '')


,chapID,para,text,paraID
0,1,1,"Каждый день над рабочей слободкой, в дымном, м...",1
1,1,2,"Вечером, когда садилось солнце, и на стеклах д...",2
2,1,3,"День проглочен фабрикой, машины высосали из му...",3
3,1,4,"По праздникам спали часов до десяти, потом люд...",4
4,1,5,"Усталость, накопленная годами, лишала людей ап...",5
...,...,...,...,...
4490,58,90,Ударили по руке.,4432
4491,58,91,"-- Только злобы накопите, безумные! На вас она...",4433
4492,58,92,Жандарм схватил ее за горло и стал душить. Она...,4434
4493,58,93,-- Несчастные...,4435


In [8]:
troeTextDf = textRegularize(libTextsDf, 0)
troeTextDf

/tmp/ipykernel_420705/3670044570.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  textDf['text'] = textDf.text.str.replace('\n|\s{2,}', '')


,text,paraID
0,Среди лесов Керженца рассеяно много одиноких м...,1
1,"Суровый характером, богатый мужик Антипа Лунёв...",2
2,"Умер он в год, когда разоряли скиты, и смерть ...",3
3,"Приехал в лес исправник с командой, и увидали ...",4
4,-- Ты! -- крикнул исправник. -- Уходи! Ломать ...,5
...,...,...
3814,"-- Совсем, кажись... башка лопнула...",3783
3815,-- Гляди -- мозг...,3784
3816,Чёрные фигуры каких-то людей выскакивали из ть...,3785
3817,"-- Ах, леший... -- тихо выговорил полицейский,...",3786


In [12]:
motherXmlDf = motherTextDf['text'].str.split(' \n', expand=True).stack().to_frame().reset_index().rename(columns={'level_0':'chapID','level_1':'para',0:'text'})
motherXmlDf = motherXmlDf.loc[~motherXmlDf.text.str.contains(r"^\W*$", regex=True)]
motherXmlDf['text'] = motherXmlDf.text.str.replace('\n|\s{2,}', '')
motherXmlDf['part'] = motherXmlDf.chapID.apply(lambda x: int('1') if x < 30 else int('2'))
motherXmlDf['chap'] = motherXmlDf.chapID.map(motherTextDf['chapID'].to_dict())
motherXmlDf['para'] = motherXmlDf['para'].apply(lambda x: x+1)
motherXmlDf['paraID'] = range(1, len(motherXmlDf)+1)
motherXmlDf = motherXmlDf.reset_index().drop('index', axis=1)

motherXmlDf

/tmp/ipykernel_420705/2287085190.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  motherXmlDf['text'] = motherXmlDf.text.str.replace('\n|\s{2,}', '')


,chapID,para,text,part,chap,paraID
0,0,1,"Каждый день над рабочей слободкой, в дымном, м...",1,1,1
1,1,1,"Вечером, когда садилось солнце, и на стеклах д...",1,1,2
2,2,1,"День проглочен фабрикой, машины высосали из му...",1,1,3
3,3,1,"По праздникам спали часов до десяти, потом люд...",1,1,4
4,4,1,"Усталость, накопленная годами, лишала людей ап...",1,1,5
...,...,...,...,...,...,...
4431,4490,1,Ударили по руке.,2,58,4432
4432,4491,1,"-- Только злобы накопите, безумные! На вас она...",2,58,4433
4433,4492,1,Жандарм схватил ее за горло и стал душить. Она...,2,58,4434
4434,4493,1,-- Несчастные...,2,58,4435


### Make XML

In [ ]:
root = etree.Element("text")
print(root.tag)
pt = ch = cn = pa = pn = 0
nameDict = motherDf.chap.to_dict()
for part in motherXmlDf.part.unique():
    print(f"Part {part}")
    root.append(etree.Element("part", n=str(part), name=str(part)))
    chapList = motherXmlDf.loc[motherXmlDf['part'] == part].chapID.unique()
    #print(chapList)
    for chap in chapList:
        #print(f"Chap {chap}")
        root[pt].append(etree.Element("chapter", n=str(cn+1), name=nameDict.get(chap)))
        paraList = motherXmlDf.loc[(motherXmlDf['part'] == part) & (motherXmlDf['chapID'] == chap)].index
        #print(paraList)
        for paragraph in paraList:
            #print(f"Paragraph {paragraph}")
            root[pt][ch].append(etree.Element("paragraph", n=str(pn+1), name=str(pa+1)))
            paraText = motherXmlDf.loc[paragraph].text
            #print(f"paraText: {paraText}")
            #print(f"pt = {pt}; ch = {ch}; paragraph = {paragraph}")
            root[pt][ch][pa].text = paraText
            pa+=1
            pn+=1
        pa=0
        ch+=1
        cn+=1
    ch=0
    pt+=1
#t+=1
#print(etree.tostring(root, pretty_print=True, xml_declaration=True))
#etree.ElementTree(root).write('../site/texts/mother.xml', pretty_print=True, xml_declaration=True, encoding='windows-1251')

In [47]:
root = etree.Element("prose")
print(root.tag)
pt = ch = pa = 1
#TestList = [x for x in bibliiaDf.test.unique()]
#for test in range(len(TestList)): 
#    root.append(etree.Element("t", n=str(t), name=TestList[test]))

for part in motherXmlDf.part.unique(): 
    print(f"Part {part}")
    ChapList = list(motherXmlDf.loc[motherXmlDf['part'] == str(part)].index)
    for chap in ChapList:
        print(f"Chap {chap}")
        root[int(part)-1].append(etree.Element("chap", n=str(ch), name=str(chap)))
        ParaList = list(motherXmlDf.loc[(motherXmlDf['part'] == part) & (motherXmlDf['chap'] == chap)].index)
        for para in ParaList:
            print(f"Para {para}")
            root[part][chap].append(etree.Element("para", n=str(pa), name=str(para)))
            para_text = motherXmlDf.loc[pa].text
            print(f"Paratext: {para_text}")
            

prose
Part 1
Part 2


### Tokens

In [4]:
motherTokenDf = pd.read_pickle('./proc/MotherTokendf.pkl')

In [5]:
motherTokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice
0,1,0,6,Каждый,1_1,1_2,det,DET,каждый,None,None,Acc,None,Masc,None,Sing,None,None,None,None
1,1,7,11,день,1_2,1_27,obl,NOUN,день,Inan,None,Acc,None,Masc,None,Sing,None,None,None,None
2,1,12,15,над,1_3,1_5,case,ADP,над,None,None,None,None,None,None,None,None,None,None,None
3,1,16,23,рабочей,1_4,1_5,amod,ADJ,рабочий,None,None,Ins,Pos,Fem,None,Sing,None,None,None,None
4,1,24,33,слободкой,1_5,1_27,obl,NOUN,слободка,Inan,None,Ins,None,Fem,None,Sing,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4436,0,6,Кто-то,1_1,1_2,nsubj,PRON,кто-то,None,None,Nom,None,None,None,None,None,None,None,None
1,4436,7,14,ответил,1_2,1_0,root,VERB,ответить,None,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Act
2,4436,15,17,ей,1_3,1_2,iobj,PRON,она,None,None,Dat,None,Fem,None,Sing,3,None,None,None
3,4436,18,25,громким,1_4,1_5,amod,ADJ,громкий,None,None,Dat,Pos,None,None,Plur,None,None,None,None


In [9]:
GetRankDf(motherTokenDf)

,n,rank
lemma,,
она,1997,1
он,1813,2
я,1120,3
мать,899,4
весь,807,5
...,...,...
прибавиться,1,11873
имущество,1,11874
еевнимательный,1,11875


In [7]:
tokenDf.token.value_counts().to_frame().reset_index().iloc[:60]

,index,token
0,и,49068
1,в,23809
2,не,19322
3,на,16869
4,я,13517
5,а,13015
6,он,12622
7,что,10842
8,с,10581
9,как,9135


In [ ]:
sentDf.sents.apply(lambda x: x.strip(r"--")).to_frame()